In [11]:
import nibabel as nib
import pandas as pd
import numpy as np

def extract_values_from_coordinates(file_paths, coordinates, example_img_path):
    """
    Extract values from NIfTI files based on MNI coordinates and store them in a Pandas DataFrame.
    
    Parameters:
        file_paths (list): List of paths to the NIfTI files.
        coordinates (list): List of dictionaries containing MNI coordinates in the form {'x': x_val, 'y': y_val, 'z': z_val}.
        example_img_path (str): Path to an example NIfTI image to get the affine matrix from.
        
    Returns:
        pd.DataFrame: DataFrame with NIfTI file paths as rows and coordinates as columns.
    """
    from nimlab import datasets as nimds
    # Load the example NIfTI file to get the affine matrix
    if example_img_path is None:
        example_img = nimds.get_img("mni_icbm152")
    else:
        example_img = nib.load(example_img_path)
    affine = example_img.affine
    inv_affine = np.linalg.inv(affine)
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(index=file_paths)
    
    for coord in coordinates:
        col_name = f"x{coord['x']}_y{coord['y']}_z{coord['z']}"
        values = []
        
        for file_path in file_paths:
            try:
                img = nib.load(file_path)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            
            data = img.get_fdata()
            
            mni_coord = np.array([coord['x'], coord['y'], coord['z'], 1])
            voxel_coord = np.dot(inv_affine, mni_coord)
            i, j, k, _ = np.round(voxel_coord).astype(int)
            
            try:
                value = data[i, j, k]
            except IndexError:
                print(f"Invalid coordinate {coord} for file {file_path}")
                value = np.nan
            
            values.append(value)
        
        df[col_name] = values
    
    return df


file_paths = List of paths to the NIfTI files.

coordinates = List of dictionaries containing MNI coordinates in the form {'x': x_val, 'y': y_val, 'z': z_val}.

example_img_path = Path to an example NIfTI image to get the affine matrix from.

In [12]:
shared_root = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo'
file_pattern = '*'
coordinates_list = [
    {'x': -26,'y': 40, 'z': -10}
]

In [13]:
from calvin_utils.file_utils.file_path_collector import glob_file_paths
unsorted_df = extract_values_from_coordinates(file_paths=glob_file_paths(shared_root, file_pattern)['paths'].to_list(),
                                coordinates = coordinates_list,
                                example_img_path=None
                                )
unsorted_df

I will search:  /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/*


,x-26_y40_z-10
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Jin2016Hippocampus_combined_func_seed_AvgR_Fz.nii,-0.080263
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Velasco1987Cm_combined_func_seed_AvgR_Fz.nii,-0.017942
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Hodaie2002Ant_combined_func_seed_AvgR_Fz.nii,0.051250
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Curkiet2020Hippocampus_combined_func_seed_AvgR_Fz.nii,-0.063084
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Herrman2018Ant_combined_func_seed_AvgR_Fz.nii,0.051353
...,...
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/JinLee2012Ant_combined_func_seed_AvgR_Fz.nii,0.045160
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Agashe2022Cm_combined_func_seed_AvgR_Fz.nii,0.001948
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Cui2023Stn_combined_func_seed_AvgR_Fz.nii,0.006759
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo/Warren2022Cm_combined_func_seed_AvgR_Fz.nii,0.024121


In [14]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis'

In [15]:
import os
unsorted_df.to_csv(os.path.join(out_dir, 'voxel_values3.csv'))